# Impurity Measures on Mushroom Dataset (placeholder)

This notebook will illustrate Information Gain, Gain Ratio and Gini
using the Mushroom Classification dataset.

Code will mirror `decision_trees/impurity_measures_mushrooms.py`.